# Car resale value prediction

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("autos.csv",encoding = "ISO-8859-1")
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   dateCrawled          371528 non-null  object
 1   name                 371528 non-null  object
 2   seller               371528 non-null  object
 3   offerType            371528 non-null  object
 4   price                371528 non-null  int64 
 5   abtest               371528 non-null  object
 6   vehicleType          333659 non-null  object
 7   yearOfRegistration   371528 non-null  int64 
 8   gearbox              351319 non-null  object
 9   powerPS              371528 non-null  int64 
 10  model                351044 non-null  object
 11  kilometer            371528 non-null  int64 
 12  monthOfRegistration  371528 non-null  int64 
 13  fuelType             338142 non-null  object
 14  brand                371528 non-null  object
 15  notRepairedDamage    299468 non-nu

# Data Preprocessing

In [5]:
df.shape

(371528, 20)

In [6]:
df['seller'].unique()

array(['privat', 'gewerblich'], dtype=object)

In [7]:
df['offerType'].unique()

array(['Angebot', 'Gesuch'], dtype=object)

In [8]:
df['abtest'].unique()

array(['test', 'control'], dtype=object)

In [9]:
df['vehicleType'].unique()

array([nan, 'coupe', 'suv', 'kleinwagen', 'limousine', 'cabrio', 'bus',
       'kombi', 'andere'], dtype=object)

In [10]:
df['gearbox'].unique()

array(['manuell', 'automatik', nan], dtype=object)

In [11]:
df['fuelType'].unique()

array(['benzin', 'diesel', nan, 'lpg', 'andere', 'hybrid', 'cng',
       'elektro'], dtype=object)

In [12]:
df['brand'].unique()

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'opel', 'seat',
       'citroen', 'honda', 'fiat', 'mini', 'smart', 'hyundai',
       'sonstige_autos', 'alfa_romeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'porsche', 'toyota', 'chevrolet',
       'dacia', 'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar',
       'daewoo', 'rover', 'land_rover', 'lada'], dtype=object)

In [13]:
df['notRepairedDamage'].unique()

array([nan, 'ja', 'nein'], dtype=object)

In [14]:
df = df.drop(['dateCrawled','name','model','powerPS','monthOfRegistration',
             'dateCreated','nrOfPictures','postalCode','lastSeen','notRepairedDamage','brand'],axis=1)
df.head()

,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,kilometer,fuelType
0,privat,Angebot,480,test,NaN,1993,manuell,150000,benzin
1,privat,Angebot,18300,test,coupe,2011,manuell,125000,diesel
2,privat,Angebot,9800,test,suv,2004,automatik,125000,diesel
3,privat,Angebot,1500,test,kleinwagen,2001,manuell,150000,benzin
4,privat,Angebot,3600,test,kleinwagen,2008,manuell,90000,diesel


In [15]:
df.isnull().sum()

seller                    0
offerType                 0
price                     0
abtest                    0
vehicleType           37869
yearOfRegistration        0
gearbox               20209
kilometer                 0
fuelType              33386
dtype: int64

In [16]:
df['vehicleType'] = df['vehicleType'].fillna(df['vehicleType'].mode()[0])
df['gearbox'] = df['gearbox'].fillna(df['gearbox'].mode()[0])
df['fuelType'] = df['fuelType'].fillna(df['fuelType'].mode()[0])

In [17]:
df.isnull().sum()

seller                0
offerType             0
price                 0
abtest                0
vehicleType           0
yearOfRegistration    0
gearbox               0
kilometer             0
fuelType              0
dtype: int64

In [18]:
df['Age'] = 2022-df['yearOfRegistration']
df = df.drop(['yearOfRegistration'],axis=1)

In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [20]:
df['seller'] = le.fit_transform(df['seller'])
df['offerType'] = le.fit_transform(df['offerType'])
df['abtest'] = le.fit_transform(df['abtest'])
df['vehicleType'] = le.fit_transform(df['vehicleType'])
df['gearbox'] = le.fit_transform(df['gearbox'])
df['fuelType'] = le.fit_transform(df['fuelType'])

In [21]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df[['kilometer']] = ss.fit_transform(df[['kilometer']])

In [22]:
df = df[['seller','offerType','abtest','vehicleType','Age','gearbox','fuelType','kilometer','price']]
df.head()

,seller,offerType,abtest,vehicleType,Age,gearbox,fuelType,kilometer,price
0,1,0,1,6,29,1,1,0.607827,480
1,1,0,1,3,11,1,3,-0.015424,18300
2,1,0,1,7,18,0,3,-0.015424,9800
3,1,0,1,4,21,1,1,0.607827,1500
4,1,0,1,4,14,1,3,-0.887975,3600


# Train test split

In [23]:
target = df['price']
x = df.drop('price',axis=1)

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,target,test_size=0.25,random_state=101)

In [25]:
X_train

,seller,offerType,abtest,vehicleType,Age,gearbox,fuelType,kilometer
53346,1,0,1,7,24,1,1,0.607827
21076,1,0,0,1,23,0,1,0.607827
322609,1,0,0,6,32,1,1,-0.638674
214931,1,0,1,6,24,0,1,0.607827
292147,1,0,0,7,19,0,3,-0.638674
...,...,...,...,...,...,...,...,...
311895,1,0,1,4,22,1,3,0.607827
136767,1,0,1,2,27,1,1,-3.007026
204614,1,0,0,5,22,1,1,0.607827
214539,1,0,0,4,20,1,1,0.607827


In [26]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(278646, 8)
(92882, 8)
(278646,)
(92882,)


# Model building using Random forest

In [27]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=3)

In [28]:
rfr.fit(X_train,y_train)

RandomForestRegressor(max_depth=3)

In [29]:
x_pred = rfr.predict(X_train)
y_pred = rfr.predict(X_test)

In [30]:
from sklearn.metrics import mean_squared_error,r2_score
print("MSE of train set:",mean_squared_error(y_train,x_pred))
print("RMSE of train set: ",np.sqrt(mean_squared_error(y_train,x_pred)))
print()
print("MSE of test set:",mean_squared_error(y_test,y_pred))
print("RMSE of test set: ",np.sqrt(mean_squared_error(y_test,y_pred)))
print()
print("Fitting on data:",r2_score(y_test,y_pred))

MSE of train set: 14941357663517.934
RMSE of train set:  3865405.239236623

MSE of test set: 5230205827159.769
RMSE of test set:  2286964.3257295834

Fitting on data: -5.067465595252745


# Model building using SVM

In [31]:
from sklearn.svm import SVR
sv = SVR()

In [ ]:
sv.fit(X_train,y_train)

In [ ]:
x_pred = sv.predict(X_train)
y_pred = sv.predict(X_test)

In [ ]:
print("MSE of train set:",mean_squared_error(y_train,x_pred))
print("RMSE of train set: ",np.sqrt(mean_squared_error(y_train,x_pred)))
print()
print("MSE of test set:",mean_squared_error(y_test,y_pred))
print("RMSE of test set: ",np.sqrt(mean_squared_error(y_test,y_pred)))
print()
print("Fitting on data:",r2_score(y_test,y_pred))

# Making Predicition

In [ ]:
from pickle import dump,load
dump(sv,open('resale.nav','wb'))

In [ ]:
value = load(open('resale.nav','rb'))
price = value.predict([[1,0,1,7,24,1,1,0.607827]])
print("Car resale value prediction: ₹",round((ss.inverse_transform([price])[0][0]),2))

# Conclusion

* seller
* offerType
* abset
* VehicleType
* Age
* gearBox
* fuelType
* kilometer

**Using the abover parameters only we are predicting the PRICE.